In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
# companiesPath = "data/Companies.txt" # Useless for this program
dataCenterPath = "data/DataCenters.txt"
dailyPwrConsPath = "data/DailyPowerConsumption.txt"

outputPath1 = "outPart1/"
outputPath2 = "outPart2/"

In [ ]:
# Define the dataframes associated with the input files
# companiesPath is useless for this application
dataCenterDF = spark.read.format("csv")\
                .option("header", True)\
                .option("inferSchema", True)\
                .load(dataCenterPath).cache()

# Register the temporary table DataCenters 
dataCenterDF.createOrReplaceTempView("DataCenters")


pwrConsDF = spark.read.format("csv")\
                .option("header", True)\
                .option("inferSchema", True)\
                .load(dailyPwrConsPath).cache()

# Register the temporary table DailyPwrCons
pwrConsDF.createOrReplaceTempView("DailyPwrCons")

In [ ]:
#########################################
# PART 1
#########################################

In [ ]:
# Select the dates associated with high power consumption
# and count the number of data centers with high power consumption for each of those dates.
# Select the dates with at least 90% of the data centers consuming at least 1000 kWh each.
res1SelectedDateDF = spark.sql("SELECT Date "\
                            + "FROM DailyPwrCons "\
                            + "WHERE kWh>=1000 "\
                            + "GROUP BY Date "\
                            + "HAVING COUNT(*)>=(SELECT COUNT(*)*0.9 "\
                            + "					 FROM DataCenters)")

In [ ]:
# Store the result
res1SelectedDateDF.write\
                .format("csv")\
                .option("header", False)\
                .save(outputPath1)

In [ ]:
#########################################
# PART 2
#########################################

In [ ]:
# Register a function that given a month returns the year
spark.udf.register("EXTRACTYEAR", \
                   lambda month: int(month.split("/")[0]), \
                   IntegerType())

In [ ]:
# Consider the power consumptions and keep only the entries related to year 2021
# and compute the power consumption for the entire year for each data center
yearlyPwrCons2021DF = spark.sql("SELECT CodDC, SUM(kWh) AS Cons2021 "\
                                + "FROM DailyPwrCons "\
                                + "WHERE EXTRACTYEAR(Date)=2021 "\
                                + "GROUP BY CodDC ")

# Register the temporary table yearlyPwrCons2021
yearlyPwrCons2021DF.createOrReplaceTempView("yearlyPwrCons2021")

In [ ]:
# Compute for each continent the number of data centers and the 
# the avg power consumption per data center in year 2021
numDCandAvgPwrConsDF = spark\
    .sql("SELECT Continent, COUNT(*) AS NumDataCenters, AVG(Cons2021) AS AvgCons2021PerDC " \
         + "FROM yearlyPwrCons2021, DataCenters " \
         + "WHERE yearlyPwrCons2021.CodDC=DataCenters.CodDC " \
         + "GROUP BY Continent").cache()

# Register the temporary table numDCandAvgPwrCons
numDCandAvgPwrConsDF.createOrReplaceTempView("numDCandAvgPwrCons")

In [ ]:
numDCandAvgPwrConsDF.show()

In [ ]:
# Compute the maximum number of data centers and the maximum avg consumption
# (see the content of the table function in the following query. It is used to compute the
# two maximum values of interest) and
# select only the continent(s) satisfying the two constraints

res2SelectedContinentsDF = spark\
            .sql("SELECT Continent " \
                 + "FROM numDCandAvgPwrCons, (SELECT Max(NumDataCenters) AS MaxDCs, Max(AvgCons2021PerDC) AS MaxAvgCons2021PerDC " \
                 + "							FROM numDCandAvgPwrCons) MaxTable " \
                 + "WHERE numDCandAvgPwrCons.NumDataCenters=MaxTable.MaxDCs " \
                 + "  AND numDCandAvgPwrCons.AvgCons2021PerDC=MaxTable.MaxAvgCons2021PerDC")

In [ ]:
# Store the result in the second output folder
res2SelectedContinentsDF.write\
                .format("csv")\
                .option("header", False)\
                .save(outputPath2)